<a href="https://colab.research.google.com/github/ebknudsen/godiva_iv/blob/notebook/cad_to_openmc_godiva_iv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

In [2]:
!conda install -y cad-to-openmc
!conda install openmc=0.15.0=dagmc_*

/bin/bash: line 1: conda: command not found


You should now have a functioning conda-environment from which you can run cad-to-openmc and OpenMC. This is done by importing the python module <code>CAD_to_OpenMC.assembly</code>.

In [3]:
import CAD_to_OpenMC.assembly as ab

ModuleNotFoundError: No module named 'CAD_to_OpenMC'

To get access to the Godiva_IV model we may simply clone a github repository where step-files containing both a simplifed version as well as a more detailed model of the Godiva IV device as described in the report "cite:Godiva_IV".

There are three step-files associated with each of the reactor models. Namely the reactor core, control rod, and burst rod. There are two control rods and a single burst rod, set in a 60 deg. symmetric pattern around the vertical axis through the reactor, center.

In the reactor experiments the control rods were used to set the overall working point of the reactor, and then the burst rod was used to reach criticality in short timespans, "bursts".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
 #clone the godiva repo to get access to the step-files.
 !git clone https://www.github.com/ebknudsen/godiva_iv

Cloning into 'godiva_iv'...
fatal: could not read Username for 'https://github.com': No such device or address


In [4]:
A=ab.Assembly(['sample_data/GIV_BR_detailed.step'])
B=ab.Assembly(['sample_data/GIV_CR_detailed.step'])
C=ab.assembly(['sample_data/GIV_core_detailed.step'])

In [ ]:
#now we run the meshing procedure on the models. At present the "regularly" best backend to use is the 'db'
A.run(backend='db', h5m_filename='GIV_BR_det.h5m')
B.run(backend='db', h5m_filename='GIV_CR_det.h5m')
C.run(backend='db', h5m_filename='GIV_core_det.h5m')

If all is well we should now have 3 files with the suffix ',h5m' in the working directory. These files contain al the vertices and triangles of a surface mesh-approximation of the reactor model.

If we examine the output from CAD_to_OpenMC we find that we have gotten reports from it about material-names, or tags, that it has attached to the various parts of the reactor,model. The material-names must coincide with the ones late inserted into the OpenMC-model.

Should you wish to do so, it is possible to "retag" parts, during the meshing process. This is done, either by supplying a python dictionary which will use regular expressions to match material tags, or a simple python list which will simply apply tags in the order they are encountered. This is useful, in the cases where no part name-tag is found in the .step-file. N.b. this procedure triggers a complete new mashing run, which can be computationally expensive (A future version of CAD_to_OpenMC will include a retagging procedure).